In [1]:
import tqdm.notebook as tqdm
import numpy as np
import scipy
import sklearn

# Spark

In [2]:
! /home/jovyan/start-hadoop.sh

jovyan
 * Starting OpenBSD Secure Shell server sshd
start-stop-daemon: unable to set gid to 0 (Operation not permitted)
   ...fail!
 * sshd is running
Starting namenodes on [localhost]
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
localhost: namenode is running as process 165.  Stop it first and ensure /tmp/hadoop-jovyan-namenode.pid file is empty before retry.
Starting datanodes
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
localhost: datanode is running as process 277.  Stop it first and ensure /tmp/hadoop-jovyan-datanode.pid file is empty before retry.
Starting secondary namenodes [6b4ace4c98df]
6b4ace4c98df: Warning: Permanently added '6b4ace4c98df' (ED25519) to the list of known hosts.
6b4ace4c98df: secondarynamenode is running as process 493.  Stop it first and ensure /tmp/hadoop-jovyan-secondarynamenode.pid file is empty before retry.
Starting resourcemanager
resourcemanager is running as process

In [3]:
# connect, context, session

import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-05-09 16:49:13,481 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


## HDFS

In [4]:
! hdfs dfs -df -h

Filesystem                Size     Used  Available  Use%
hdfs://localhost:9000  251.0 G  291.6 M    225.9 G    0%


In [5]:
! hdfs dfs -ls /

Found 2 items
drwxrwx---   - root   supergroup          0 2023-05-09 16:49 /tmp
drwxr-xr-x   - jovyan supergroup          0 2023-05-09 16:49 /user


In [9]:
# were downloaded from chat in tg
! hdfs dfs -put page_views_sample.csv
! hdfs dfs -put documents_topics.csv

put: `page_views_sample.csv': File exists
put: `documents_topics.csv': File exists


In [11]:
views = se.read.csv("page_views_sample.csv", header=True)
topics = se.read.csv("documents_topics.csv", header=True)

## Top 10 most visited document_ids in the page_views_sample log

In [12]:
grouped_and_counted_views = views.groupBy("document_id").count()
first_10_rows = grouped_and_counted_views.orderBy("count", ascending=False).limit(10)
ans1 = first_10_rows.rdd.map(lambda row: int(row.document_id)).collect()

In [13]:
ans1

[1811567,
 234,
 42744,
 1858440,
 1780813,
 60164,
 1790442,
 1877626,
 1821895,
 732651]

## How many users have at least 2 different traffic_from pyspark.sql.functions import desc, countDistinctsources in the page_views_sample log (note the value is not a count, it's an encoded enum)

In [15]:
from pyspark.sql.functions import countDistinct

In [17]:
grouped = views.groupBy("uuid")
aggregated_by_distinct = grouped.agg(distinct().count()countDistinct("traffic_source").alias("dts"))
filtered = aggregatged_by_distinct.filter(aggregatged_by_distinct.dts >= 2)
ans2 = filtered.count()

In [18]:
ans2

98080

## Top 10 most visited topic_ids in page_views_sample log (use documents_topics table)

In [19]:
joined = views.join(documents_topics, on="document_id", how="inner")
grouped_and_counted = joined.groupBy("topic_id").count()
sorted_and_limited = grouped_and_counted.orderBy("count", ascending=False).limit(10)
ans3 = sorted_and_limited.rdd.map(lambda row: int(row.topic_id)).collect()

In [20]:
ans3

[20, 16, 216, 136, 140, 143, 36, 97, 8, 269]

## Answer

In [24]:
import json

In [25]:
answer = {"top_10_documents": ans1,
          "users": ans2,
          "top_10_topics": ans3}

In [26]:
with open("answer.json", "w") as f:
    json.dump(answer, f)

In [29]:
! curl -F file=@answer.json "51.250.54.133:80/MDS-LSML1/Vamiko/w4/1"

1.0
Well done!
